<a href="https://colab.research.google.com/github/ssimpore/IBM-R/blob/master/creditPrevision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler,MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer,make_column_selector
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pickle

In [11]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/home_loan_data.csv")
df = data.copy()

In [12]:
X_ = df.drop(columns=['Loan_ID','Loan_Status'])
y_ = df[['Loan_Status']]

In [13]:
def buildDataPreprocessing(arg: pd.DataFrame) -> pd.DataFrame:
    arg = arg.copy()

    cat_selector = make_column_selector(dtype_include=object)
    num_selector = make_column_selector(dtype_include=np.number)

    numeric_features = num_selector(arg)
    categorical_features = cat_selector(arg)

    numeric_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            #('scaler', StandardScaler())
            ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', fill_value='missing')),
        ('encoder', OneHotEncoder(handle_unknown='ignore',drop='first'))])


    preprocessor = ColumnTransformer(transformers=[
          ('num', numeric_transformer, numeric_features),
          ('cat', categorical_transformer, categorical_features)
          ])

    df_ = preprocessor.fit_transform(arg)

    #cat_col_out = preprocessor.named_transformers_['cat']['encoder'].get_feature_names(categorical_features)

    cat_col_out = preprocessor.named_transformers_['cat']['encoder'].get_feature_names_out(categorical_features)

    df_ = pd.DataFrame(df_,columns=numeric_features+list(cat_col_out))

    return df_


In [14]:
X_train_,X_test_,y_train_,y_test_= train_test_split(X_,y_,test_size=0.2,random_state=6)

X_train = buildDataPreprocessing(X_train_)
y_train = buildDataPreprocessing(y_train_)
X_test = buildDataPreprocessing(X_test_)
y_test = buildDataPreprocessing(y_test_)


In [15]:
pipe = Pipeline(steps=[('regressor',  LogisticRegression())])
pipe.fit(X_train,y_train)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('regressor', LogisticRegression())])

In [16]:
pred =pipe.predict(X_test)
pred

score = accuracy_score(y_test,pred)
score

0.8373983739837398

In [17]:
inputsDataDict = {'Gender':'Male',
              'Married':'Yes',
              'Dependents':'3+',
              'Education':'Graduate',
              'Self_Employed':'No',
              'ApplicantIncome':3000,
              'CoapplicantIncome':0,
              'LoanAmount':2000.0,
              'Loan_Amount_Term':360.0,
              'Credit_History':1,
              'Property_Area':'Urban'}


def getInputsData(df_train :pd.DataFrame, inputsDataDict:dict):
    arg = pd.DataFrame(inputsDataDict,index=[0])
    arg = pd.concat([arg,df_train],axis=0)
    arg = buildDataPreprocessing(arg)
    arg = arg[:1]
    return arg

X_pred = getInputsData(X_train_,inputsDataDict)


pipe.predict(X_pred)

array([0.])

In [18]:
#Save the model
pickle.dump(pipe,open('prevision_credit.pkl','wb'))